# 第二章第三节：控制流（Control Flow）


## 本节的目的
到目前为止，Chisel中的硬件实现都还能和软件对应上。
在控制流中，硬件和软件将开始会有很大的不同。
这一节中会介绍软件生成器中的控制流和硬件中的控制流。
例如重新连接Chisel中的线（wire）会发生什么？
怎样定义一个具有两个以上输入的多路复用器？
完成这一节后，您就可以回答这些问题。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [2]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

import chisel3._

import chisel3.util._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# 最终连接的语义

<span style="color:blue">**例子：重新赋值**</span><br>
如前所述，Chisel允许您使用`:=`运算符来连接组件。
由于种种原因，您可以多次将信号连接到同一组件上。
发生这种情况时，最后一个语句将会获胜。

In [3]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

// Chisel 代码: 声明一个新的测试
class LastConnectTester(c: LastConnect) extends PeekPokeTester(c) {
  expect(c.io.out, 4)  // 断言：输出将会是最后一次连接 4
}

//  测试 LastConnect
val works = Driver(() => new LastConnect) {
  c => new LastConnectTester(c)
}
assert(works)        // Scala 代码: 如果 works == false 的话会抛出异常
println("SUCCESS!!") // Scala 代码: 到这里的话表示测试通过

[info] [0.001] Elaborating design...
[info] [0.751] Done elaborating.
Total FIRRTL Compile Time: 385.8 ms
Total FIRRTL Compile Time: 14.8 ms
End of dependency graph
Circuit state created
[info] [0.002] SEED 1562046535998
test cmd2HelperLastConnect Success: 1 tests passed in 5 cycles taking 0.019492 seconds
[info] [0.008] RAN 0 CYCLES PASSED
SUCCESS!!


defined class LastConnect
defined class LastConnectTester
works: Boolean = true

---
# `when`，`elsewhen`，和`otherwise`

Chisel中条件逻辑的主要依赖于`when`，`elsewhen`和`otherwise`结构来实现。
类似下面这样：
```scala
when(someBooleanCondition) {
  // 当someBooleanCondition条件为真时的电路
}.elsewhen(someOtherBooleanCondition) {
  // 当someBooleanCondition条件为假，但是someOtherBooleanCondition为真时的电路
}.otherwise {
  // 上面所有条件都不为真时的电路
}
```

`when`，`elsewhen`，和`otherwise`必须以上述顺序出现，但除`when`之外，后面两个中的任何一个都可以被省略。
根据需要，也可以包含更多的`elsewhen`语句。
当任何语句为真时，该构造被终止。
在三个语句块中可以包含任何语句，甚至是嵌套包含`when`。

和Scala中的`if`**不一样**，`when`中的语句块不能返回值。
例如下面的情况是非法的：
```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
上面的语句**不能**工作。我们将在*Wires*一节讨论解决方案。

<span style="color:blue">**例子：Chisel的条件语句**</span><br>
下面是一个使用了`when`语句模块`Module`的例子。

In [4]:
// Max3返回三个输入中最大的一个
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 > io.in2 && io.in1 > io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 > io.in1 && io.in2 > io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// 测试正确的返回三个输入中最大的一个
class Max3Tester(c: Max3) extends PeekPokeTester(c) {
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)  
  poke(c.io.in3, 2)  
  expect(c.io.out, 6)  // 第一个输入应该是最大的
  poke(c.io.in2, 7)  
  expect(c.io.out, 7)  // 第二个输入应该是最大的
  poke(c.io.in3, 11)  
  expect(c.io.out, 11) // 第三个输入应该是最大的
  poke(c.io.in3, 3)  
  expect(c.io.out, 7)  // 测试递减序列
}

// 测试 Max3
val works = Driver(() => new Max3) {
  c => new Max3Tester(c)
}
assert(works)        // Scala 代码：如果 works == false 会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

[info] [0.000] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 57.9 ms
Total FIRRTL Compile Time: 23.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1562046538403
test cmd3HelperMax3 Success: 4 tests passed in 5 cycles taking 0.013157 seconds
[info] [0.009] RAN 0 CYCLES PASSED
SUCCESS!!


defined class Max3
defined class Max3Tester
works: Boolean = true

---
# `Wire`结构

让我们回到上面`when`不返回值的限制。
Chisel中的`Wire`结构是解决这个问题的方法之一。
`Wire`定义了一个可以出现在语句右侧的电路器件，它也可以放在语句的左侧，之后紧跟`:=`操作符。

<span style="color:blue">**例子：使用线Wire的4个输入的排序电路**</span><br>

为了说明这一点，我们来写一个用于排序的组合电路。它有四个输入和四个输出。如下图中所示，当左边的值小于右边的值时，数据会循着红线到下一行；而当左边的值大于右边时，数据会循着黑线，从而交换这两个值。

![Sort4](images/Sorter4.png)

该图显示了一系列名称以*row*开头的单元，我们将使用`Wire`来构建这些单元，并放置上面的拷贝或交换的结果。这个代码非常冗长，但之后您会看到缩减版。

In [5]:
/** Sort4 将4个输入排序，并输出4个数 */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // 保留头两个元素
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // 交换头两个元素
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // 保留最后两个元素
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // 交换最后两个元素
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // 保留中间两个元素
    row22 := row12
  }.otherwise {
    row21 := row12            // 交换中间两个元素
    row22 := row11
  }

  val row20 = Wire(UInt(16.W))
  val row23 = Wire(UInt(16.W))
  when(row10 < row13) {
    row20 := row10            // 保留中间两个元素
    row23 := row13
  }.otherwise {
    row20 := row13            // 交换中间两个元素
    row23 := row10
  }

  when(row20 < row21) {
    io.out0 := row20            // 保留头两个元素
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // 交换头两个元素
    io.out1 := row20
  }

  when(row22 < row23) {
    io.out2 := row22            // 保留头两个元素
    io.out3 := row23
  }.otherwise {
    io.out2 := row23            // 交换头两个元素
    io.out3 := row22
  }
}

// 检查输入被正确的排序
class Sort4Tester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 3)
  expect(c.io.out1, 6)
  expect(c.io.out2, 9)
  expect(c.io.out3, 12)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)
    
  poke(c.io.in0, 13)
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)
}

// 测试
val works = iotesters.Driver(() => new Sort4) {
c => new Sort4Tester(c)
}
assert(works) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了


[info] [0.000] Elaborating design...
[info] [0.018] Done elaborating.
Total FIRRTL Compile Time: 85.6 ms
Total FIRRTL Compile Time: 36.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1562046539525
test cmd4HelperSort4 Success: 12 tests passed in 5 cycles taking 0.020994 seconds
[info] [0.015] RAN 0 CYCLES PASSED
SUCCESS!!


defined class Sort4
defined class Sort4Tester
works: Boolean = true

下面是一个更详尽的测试，使用了Scala的`List`功能。您可以在之后的模块中看到更多`List`的函数。

In [6]:
// 检查四个数的所有排列组合的排序
class BetterSort4Tester(c: Sort4) extends PeekPokeTester(c) {
  List(1, 2, 3, 4).permutations.foreach { case i0 :: i1 :: i2 :: i3 :: Nil =>
    println(s"Sorting $i0 $i1 $i2 $i3")
    poke(c.io.in0, i0)
    poke(c.io.in1, i1)
    poke(c.io.in2, i2)
    poke(c.io.in3, i3)
    expect(c.io.out0, 1)
    expect(c.io.out1, 2)
    expect(c.io.out2, 3)
    expect(c.io.out3, 4)
  }
}

// 测试
val works = iotesters.Driver(() => new Sort4) {
c => new BetterSort4Tester(c)
}
assert(works) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

[info] [0.000] Elaborating design...
[info] [0.078] Done elaborating.
Total FIRRTL Compile Time: 66.5 ms
Total FIRRTL Compile Time: 37.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1562046540126
[info] [0.016] Sorting 1 2 3 4
[info] [0.022] Sorting 1 2 4 3
[info] [0.023] Sorting 1 3 2 4
[info] [0.026] Sorting 1 3 4 2
[info] [0.028] Sorting 1 4 2 3
[info] [0.032] Sorting 1 4 3 2
[info] [0.034] Sorting 2 1 3 4
[info] [0.036] Sorting 2 1 4 3
[info] [0.038] Sorting 2 3 1 4
[info] [0.039] Sorting 2 3 4 1
[info] [0.042] Sorting 2 4 1 3
[info] [0.043] Sorting 2 4 3 1
[info] [0.044] Sorting 3 1 2 4
[info] [0.046] Sorting 3 1 4 2
[info] [0.047] Sorting 3 2 1 4
[info] [0.048] Sorting 3 2 4 1
[info] [0.049] Sorting 3 4 1 2
[info] [0.050] Sorting 3 4 2 1
[info] [0.051] Sorting 4 1 2 3
[info] [0.052] Sorting 4 1 3 2
[info] [0.054] Sorting 4 2 1 3
[info] [0.056] Sorting 4 2 3 1
[info] [0.057] Sorting 4 3 1 2
[info] [0.058] Sorting 4 3 2 1
test cmd4HelperSort4 Success: 96 tes

defined class BetterSort4Tester
works: Boolean = true

---
# 练习

<span style="color:red">**练习：多项式电路**</span><br>
写一个模块来计算下面的多项式：
- $x^2 - 2x + 1$
- $2x^2 + 6x + 3$
- $4x^2 - 10x -5$


选择器的一个输入用来决定所要计算的多项式。使用`Wire`使得$x^2$计算在模块中只出现一次，从而输出只有一个连接。

首先让我们使用测试驱动开发（TDD），并使用Scala编写测试。完成下面这些函数，使得其通过后面的断言。这并不是一个详尽的测试，而只是一个简单的完整性检查。

In [ ]:
def poly0(x: Int): Int = ???
def poly1(x: Int): Int = ???
def poly2(x: Int): Int = ???

assert(poly0(0) == 1)
assert(poly1(0) == 3)
assert(poly2(0) == -5)

assert(poly0(1) == 0)
assert(poly1(1) == 11)
assert(poly2(1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly0(x: Int): Int = x\*x - 2\*x + 1
def poly1(x: Int): Int = 2\*x\*x + 6\*x + 3
def poly2(x: Int): Int = 4\*x\*x - 10\*x - 5
</pre></article></div></section></div>

为了使测试更容易，让我们创建一个类似于后面要完成的硬件模块的函数。使用Scala的`if`语句并根据`select`输入来选择多项式。

In [ ]:
def poly(select: Int, x: Int): Int = {
  ???
}

assert(poly(1, 0) == 3)
assert(poly(1, 1) == 11)
assert(poly(2, 1) == -11)

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
def poly(select: Int, x: Int): Int = {
  if(select == 0) {
    poly0(x)
  }
  else if(select == 1) {
    poly1(x)
  }
  else {
    poly2(x)
  }
}
</pre></article></div></section></div>

看起来这个测试计算的值是正确的。现在使用以下面的代码为模板来实现您的电路：

In [ ]:
// 计算多项式
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(2.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val result = Wire(SInt(32.W))  
  val square = Wire(SInt(32.W))  
  
  ???

  io.fOfX := result  
}

// 检查计算的正确性
class PolynomialTester(c: Polynomial) extends PeekPokeTester(c) {
  for(x <- 0 to 20) {
    for(select <- 0 to 2) {
      poke(c.io.select, select)
      poke(c.io.x, x)
      expect(c.io.fOfX, poly(select, x))
    }
  }
}

// 测试多项式
val works = Driver(() => new Polynomial) {
  c => new PolynomialTester(c)
}
assert(works) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-3" />
<label for="check-3"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  square := io.x \* io.x
  when(io.select === 0.U) {
    result := (square - (2.S \* io.x)) + 1.S
  }.elsewhen(io.select === 1.U) {
    result := (2.S \* square) + (6.S \* io.x) + 3.S
  }.otherwise {
    result := (4.S \* square) - (10.S \* io.x) - 5.S
  }
</pre></article></div></section></div>

<span style="color:red">**练习：有限状态机**</span><br>

使用卡诺图（Karnaugh map）来优化状态机的逻辑非常繁琐，这可以通过综合工具来解决。并且它还会产生不直观且难以理解的代码。因此，我们将使用Chisel控制流和最后的连接语义编写一个更简单易懂的。

研究生们在他们的学习生涯中会经历四种状态：空闲，编码，写作和毕业。这些状态根据三个输入来进行转换：咖啡，他们提出的想法，来自导师的压力。一旦他们毕业，他们就会回到空闲状态。下面的FSM图显示了这些状态和状态之间的转换。任何未标记的转换（即，当没有输入时）会将研究生返回到空闲状态而不是保持在当前状态。输入优先级是 咖啡（coffee） > 想法（idea） > 压力（pressure），所以如果处于空闲状态并同时接收到咖啡和压力时，研究生将转换到编码（Coding）状态。

<img src="images/fsm.png" width="500" />

首先，我们将构建一个模型来测试我们的硬件。在下面的代码中完成我们的状态机的功能。它有四个输入，输出是下一个状态。状态图已经提供了，您可以通过类似这样`states("grad")`来访问它。

In [ ]:
// 状态图
def states = Map("idle" -> 0, "coding" -> 1, "writing" -> 2, "grad" -> 3)

// 生活就是由一堆问号构成
def gradLife (state: Int, coffee: Boolean, idea: Boolean, pressure: Boolean): Int = {
  var nextState = states("idle")
  ???
  nextState
}

// 一些完整性测试
(0 until states.size).foreach{ state => assert(gradLife(state, false, false, false) == states("idle")) }
assert(gradLife(states("writing"), true, false, true) == states("writing"))
assert(gradLife(states("idle"), true, true, true) == states("coding"))
assert(gradLife(states("idle"), false, true, true) == states("idle"))
assert(gradLife(states("grad"), false, false, false) == states("idle"))

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-4" />
<label for="check-4"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  if (state == states("idle")) {
    if      (coffee) { nextState = states("coding") }
    else if (idea) { nextState = states("idle") }
    else if (pressure) { nextState = states("writing") }
  } else if (state == states("coding")) {
    if      (coffee) { nextState = states("coding") } 
    else if (idea || pressure) { nextState = states("writing") }
  } else if (state == states("writing")) {
    if      (coffee || idea) { nextState = states("writing") }
    else if (pressure) { nextState = states("grad") }
  }
</pre></article></div></section></div>

由于我们尚未学习时序逻辑，当前状态是模块的输入，而下一个状态是模块的输出，就像之前的`gradLife`函数一样。现在用让我们用Chisel来实现这个状态机，并让它通过测试。 Chisel为我们提供了一个方便的状态映射函数（来实现状态的值和名称之间的映射），称为`Enum`。要使用这些状态，请将它们视为类似`UInt`的字面值（literal）。记住，硬件相等使用的是三个等号！

In [ ]:
// 生活更艰难了
class GradLife extends Module {
  val io = IO(new Bundle {
    val state = Input(UInt(2.W))
    val coffee = Input(Bool())
    val idea = Input(Bool())
    val pressure = Input(Bool())
    val nextState = Output(UInt(2.W))
  })
    
  val idle :: coding :: writing :: grad :: Nil = Enum(4)
  
  io.nextState := idle
  ???
}

// 检查硬件和软件模型一致
class GradLifeSim(c: GradLife) extends PeekPokeTester(c) {
  for (state <- 0 to 3) {
    for (coffee <- List(true, false)) {
      for (idea <- List(true, false)) {
        for (pressure <- List(true, false)) {
          poke(c.io.state, state)
          poke(c.io.coffee, coffee)
          poke(c.io.idea, idea)
          poke(c.io.pressure, pressure)
          expect(c.io.nextState, gradLife(state, coffee, idea, pressure))
        }
      }
    }
  }
}

// 测试
val works = Driver(() => new GradLife) {c => new GradLifeSim(c)}
assert(works) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-5" />
<label for="check-5"><strong>答案</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  when (io.state === idle) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea) { io.nextState := idle }
    .elsewhen (io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === coding) {
    when      (io.coffee) { io.nextState := coding } 
    .elsewhen (io.idea || io.pressure) { io.nextState := writing }
  } .elsewhen (io.state === writing) {
    when      (io.coffee || io.idea) { io.nextState := writing }
    .elsewhen (io.pressure) { io.nextState := grad }
  }
</pre></article></div></section></div>

---
# 本节结束!

[返回顶部](#top)